# Topical Expert Model

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import operator
import os
print(os.getcwd())
print(os.listdir('/Users/Brandon/Yelp Dataset/dataset'))
import json
import glob
from datetime import datetime
from pandas import HDFStore,DataFrame
import operator
import pylab as pl

import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
import matplotlib

/Users/Brandon/Yelp Dataset/Yelp-FA17/local-elites/workspace
['.DS_Store', 'business.json', 'checkin.json', 'photos.json', 'review.json', 'review5000.json', 'tip.json', 'user.json', 'user5000.json']


In [ ]:
#Copied from Kenta
def load_json_to_df(datapass):
    '''
    Load the json file and parse the file to pandas dataframe format
    
    Input:
        datapass(str) : directory to the json file
    Output:
        df(dataframe) : pandas dataframe object
    '''
    
    data = [] 
    with open(datapass) as data_file: 
        for f in data_file:
            data.append(json.loads(f))
    df = pd.DataFrame(data)
    return df

LOAD THE DATA

In [ ]:
%%time
import_file = "/Users/Brandon/Yelp Dataset/dataset/review.json"
review = load_json_to_df(import_file)

In [ ]:
%%time
import_file = "/Users/Brandon/Yelp Dataset/dataset/business.json"
business = load_json_to_df(import_file)

In [ ]:
%%time
import_file = "/Users/Brandon/Yelp Dataset/dataset/user.json"
user = load_json_to_df(import_file)

Sort by category

In [ ]:
def categoryFind(df, cat):
    return [cat in i for i in df['categories']]

def category_counts(df):
    all_cats = []
    for i in df['categories']:
        for j in i:
            all_cats.append(j)
    categories = set(all_cats)
    category_counts = {}
    for cat in categories:
        category_counts[cat] = all_cats.count(cat)
    return len(categories), category_counts

def top_categories(counts, n):
    top_categories = sorted(counts.keys(), key=(lambda k: counts[k]),reverse=True)[:n]
    return top_categories

In [ ]:
num_cats, cat_counts = category_counts(business)
top_cats = top_categories(cat_counts, 15)
display(top_cats)

In [ ]:
%%time
businessReviews = pd.merge(review, business,on='business_id',right_index=True,sort=False)[['business_id', 'cool', 'funny', 'review_id', 'stars_x', 'text', 'useful', 'user_id', 'categories', 'review_count', 'stars_y']]
bus_rev_users = pd.merge(businessReviews, user,on='user_id',right_index=True,sort=False)[['business_id', 'cool_x', 'funny_x', 'review_id', 'stars_x', 'text', 'useful_x', 'user_id', 'categories', 'stars_y', 'average_stars', 'cool_y', 'elite', 'fans', 'friends', 'name', 'review_count_y', 'yelping_since']]

In [ ]:
display(len(businessReviews))
display(len(bus_rev_users))
display(list(businessReviews))
display(list(bus_rev_users))

## Feature Extraction From Yelp Data

In [ ]:
def category_choose(df, categories):
    cats = np.array([True] * len(df['categories']))
    for cat in categories:
        cats= cats & np.array(categoryFind(df,cat))
    return df[cats]

def numCategoryReviews(businessReviews):
    return businessReviews[['business_id','user_id']].groupby('user_id',as_index=False).count()

def average_star_category_rating(businessReviews):
    return businessReviews[['user_id','stars_x']].groupby('user_id',as_index=False).mean()

def std_star_category_rating(businessReviews):
    busRev = businessReviews[['user_id','stars_x']].groupby('user_id',as_index=False).agg(np.var)
    busRev['stars_x'] = busRev['stars_x'].pow(1./2)
    return busRev

def funny_useful_cool(businessReviews):
    return businessReviews[['user_id','funny','useful','cool']].groupby('user_id',as_index=False).sum()

def months_yelping(businessReviews):
    user_dates = businessReviews[['user_id','yelping_since']].drop_duplicates()
    curr_date = datetime(2017,10,17)
    uyelp_dates = [datetime.strptime(i, "%Y-%m-%d") for i in user_dates['yelping_since']]
    months_yelping = [(curr_date.year - i.year) * 12 + curr_date.month - i.month for i in uyelp_dates]
    user_dates['yelping_since'] = months_yelping
    return user_dates

def get_elite_users(df):
    temp = []
    for i in df['elite'].index.values:
        if df['elite'][i]:
            temp.append(i)
    return temp, df.loc[temp]

## Define the first feature model to be analyzed
Features: Total Reviews by User, Number of Category Reviews, Average Rating in Category, Std Dev of Ratings in Category, Funny, Useful, and Cool Votes, Number of Months since joining Yelp.

In [ ]:
def feature1(df, df1, categories):
    if( not categories == "All"):
        df = category_choose(df, categories)
        df1 = category_choose(df1, categories)
    tot_reviews = df1[['user_id','review_count_y']]
    tot_reviews.columns = ['user_id','Total Reviews by User']
    catRev = numCategoryReviews(df)
    catRev.columns = ['user_id','Num Category Reviews']
    averageCat = average_star_category_rating(df)
    averageCat.columns = ['user_id','Average Rating in Category']
    stdCat = std_star_category_rating(df)
    stdCat.columns = ['user_id','Std Dev of Ratings in Category']
    fuc = funny_useful_cool(df)
    monthsYelp = months_yelping(df1)
    monthsYelp.columns = ['user_id', 'Months Yelping']
    is_elite = df1[['user_id','elite']]
    features = tot_reviews.merge(catRev,on='user_id').merge(averageCat,on='user_id').merge(stdCat,on='user_id').merge(fuc,on='user_id').merge(monthsYelp,on='user_id').merge(is_elite,on='user_id')
    features = features.drop_duplicates('user_id')
    features.index = range(len(features))
    elite_ind, elite_users = get_elite_users(features)
    expertClassifier = [0]*len(features)
    for i in elite_ind:
        expertClassifier[i]=1
    features = pd.concat([features, pd.DataFrame(expertClassifier,columns=['is_expert'])],axis=1)
    return features.fillna(0)

# Feature 2 has all of feature 1 plus the number of friends for each user
def feature2(df, df1, categories, features1 = pd.DataFrame()):
    if features1.empty:
        features1= feature1(df,df1,categories)
    
    friends = pd.DataFrame([len(i) for i in df1['friends']], columns = ['Num_Friends'])
    users = pd.DataFrame(df1['user_id'].values,columns=['user_id'])
    num_friends = pd.concat([users, friends],axis=1)
    features1= features1.merge(num_friends,on='user_id').drop_duplicates('user_id')
    features1.index = range(len(features1))
    return features1
    

In [ ]:
%%time
# Lets do places with categories Restaurant
categories = ['Restaurants']
feature_set_1 = feature1(businessReviews, bus_rev_users, categories)

In [ ]:
feature_set_2 = feature2(businessReviews, bus_rev_users, categories, feature_set_1)
feature_set_2

In [ ]:
display(len(feature_set_1))
display(feature_set_1.head(5))

## Classifier Setups (Sklearn Library)*

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import tree

In [ ]:
'''Runs the model specified by the clf (classifier), and features.
Features is assumed to have an 'is_expert' column which gives the classifications'''
def run_model(clf, features):
    train, test = train_test_split(X)
    train_classifier = train['is_expert'].values
    test_classifier = test['is_expert'].values
    train = train.drop('is_expert',axis=1)
    test = test.drop('is_expert', axis=1)
    clf.fit(train, train_classifier)
    rf_pred = clf.predict(test)
    model_perf= {'Model_Score' : clf.score(test, test_classifier),
                'Predictions' : rf_pred,
                'Actual' : test_classifier,
                'Prediction_Probabilities' : clf.predict_proba(test),
                'Total_Tested' : len(rf_pred),
                'Num_Experts_Predicted' : sum(rf_pred),
                'Num_Experts_Actual' : sum(test_classifier),
                'Num_Experts_Training' : sum(train_classifier)} 
    return model_perf

'''Runs the model n times, and prints out a dictionary with the statistics'''
def bootstrap_model(clf, features, n):
    models = []
    for i in range(1,n):
        models.append(run_model(clf, features))
    return models

'''Gets statistics from the bootstrap list of dictionaries'''
def boot_statistics(models):
    stats = []
    mean_model_score = np.mean([i['Model_Score'] for i in models])
    expert_pred_percentage = [i['Num_Experts_Predicted']/i['Num_Experts_Actual'] for i in models]
    mn = np.array((1.0 - np.array(mean_model_score))) * np.array(models[0]['Total_Tested'])
    stats = {'Mean_Wrong_Predictions' : round(mn),
             'Mean_Model_Score' : mean_model_score, 
             'Expert_Prediction_Percentage' : expert_pred_percentage}
    return stats

def graph_from_statistics(stats):
    return 0

 #### Drop unnessary columns from the features table (call it X).

In [ ]:
X = feature_set_1.drop('user_id',axis=1).drop('elite',axis=1)
X2 = feature_set_2.drop('user_id',axis=1).drop('elite',axis=1)

## Run a random forest classifier

In [42]:
clf_RF = RandomForestClassifier(max_depth=3)

display(run_model(clf_RF,X))
display(run_model(clf_RF,X2))

RF_bootstrap = bootstrap_model(clf_RF, X, 10)
RF2_bootstrap = bootstrap_model(clf_RF, X2, 10)

{'Actual': array([0, 0, 0, ..., 0, 0, 0]),
 'Model_Score': 0.95849001603264827,
 'Num_Experts_Actual': 12563,
 'Num_Experts_Predicted': 7007,
 'Num_Experts_Training': 38116,
 'Prediction_Probabilities': array([[ 0.98667353,  0.01332647],
        [ 0.96662081,  0.03337919],
        [ 0.98667353,  0.01332647],
        ..., 
        [ 0.97008213,  0.02991787],
        [ 0.97969029,  0.02030971],
        [ 0.98667353,  0.01332647]]),
 'Predictions': array([0, 0, 0, ..., 0, 0, 0]),
 'Total_Tested': 205830}

{'Actual': array([0, 0, 0, ..., 0, 0, 0]),
 'Model_Score': 0.95146965942768302,
 'Num_Experts_Actual': 12761,
 'Num_Experts_Predicted': 4292,
 'Num_Experts_Training': 37918,
 'Prediction_Probabilities': array([[ 0.96834861,  0.03165139],
        [ 0.97960301,  0.02039699],
        [ 0.98208123,  0.01791877],
        ..., 
        [ 0.97175066,  0.02824934],
        [ 0.7813324 ,  0.2186676 ],
        [ 0.97960301,  0.02039699]]),
 'Predictions': array([0, 0, 0, ..., 0, 0, 0]),
 'Total_Tested': 205830}

## Run a gaussian naive bayes classifier

In [43]:
from sklearn.naive_bayes import GaussianNB
clf_NB = GaussianNB()

display(run_model(clf_NB,X))
display(run_model(clf_NB,X2))

NB_bootstrap = bootstrap_model(clf_NB, X, 10)
NB2_bootstrap = bootstrap_model(clf_NB, X2, 10)

{'Actual': array([0, 1, 0, ..., 0, 0, 0]),
 'Model_Score': 0.9563960549968421,
 'Num_Experts_Actual': 12577,
 'Num_Experts_Predicted': 8350,
 'Num_Experts_Training': 38102,
 'Prediction_Probabilities': array([[  9.99999920e-01,   8.03111762e-08],
        [  9.99999915e-01,   8.48009705e-08],
        [  9.99999989e-01,   1.09528665e-08],
        ..., 
        [  9.99999967e-01,   3.32475099e-08],
        [  9.99999979e-01,   2.06560250e-08],
        [  9.99998813e-01,   1.18724482e-06]]),
 'Predictions': array([0, 0, 0, ..., 0, 0, 0]),
 'Total_Tested': 205830}

{'Actual': array([0, 0, 0, ..., 0, 0, 0]),
 'Model_Score': 0.95621143662245545,
 'Num_Experts_Actual': 12776,
 'Num_Experts_Predicted': 8659,
 'Num_Experts_Training': 37903,
 'Prediction_Probabilities': array([[  9.99999964e-01,   3.56212067e-08],
        [  9.99999988e-01,   1.22353807e-08],
        [  1.00000000e+00,   2.62261638e-10],
        ..., 
        [  9.99999976e-01,   2.42089574e-08],
        [  9.99999990e-01,   9.57277537e-09],
        [  9.99999722e-01,   2.77819398e-07]]),
 'Predictions': array([0, 0, 0, ..., 0, 0, 0]),
 'Total_Tested': 205830}

## Run a decision tree classifier

In [44]:
clf_DT = tree.DecisionTreeClassifier()

display(run_model(clf_DT, X))
display(run_model(clf_DT, X2))

DT_bootstrap = bootstrap_model(clf_DT, X, 10)
DT2_bootstrap = bootstrap_model(clf_DT, X2, 10)

{'Actual': array([0, 0, 1, ..., 0, 0, 0]),
 'Model_Score': 0.95293203128795612,
 'Num_Experts_Actual': 12558,
 'Num_Experts_Predicted': 12830,
 'Num_Experts_Training': 38121,
 'Prediction_Probabilities': array([[ 1.,  0.],
        [ 1.,  0.],
        [ 0.,  1.],
        ..., 
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.]]),
 'Predictions': array([0, 0, 1, ..., 0, 0, 0]),
 'Total_Tested': 205830}

{'Actual': array([1, 0, 1, ..., 0, 0, 0]),
 'Model_Score': 0.95283000534421614,
 'Num_Experts_Actual': 12541,
 'Num_Experts_Predicted': 12872,
 'Num_Experts_Training': 38138,
 'Prediction_Probabilities': array([[ 1.,  0.],
        [ 1.,  0.],
        [ 0.,  1.],
        ..., 
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.]]),
 'Predictions': array([0, 0, 1, ..., 0, 0, 0]),
 'Total_Tested': 205830}

In [45]:
DT_stats = boot_statistics(DT_bootstrap)
NB_stats = boot_statistics(NB_bootstrap)
RF_stats = boot_statistics(RF_bootstrap)
#
DT2_stats = boot_statistics(DT2_bootstrap)
NB2_stats = boot_statistics(NB2_bootstrap)
RF2_stats = boot_statistics(RF2_bootstrap)

display(DT_stats, DT2_stats)
display(NB_stats, DT2_stats)
display(RF_stats, RF2_stats)
y = [DT_stats['Mean_Model_Score'],NB_stats['Mean_Model_Score'],RF_stats['Mean_Model_Score']]
N = len(y)
x = range(N)
width = 1/1.5
plt.bar(x, y, width)

fig = plt.gcf()

{'Expert_Prediction_Percentage': [1.0126333961079723,
  1.0196440025657472,
  1.0035957164074103,
  1.0159069325735992,
  1.0181314330958036,
  1.0156262393908146,
  0.99968592964824121,
  0.99338908801274395,
  1.0007049424297016],
 'Mean_Model_Score': 0.95309883560867392,
 'Mean_Wrong_Predictions': 9654.0}

{'Expert_Prediction_Percentage': [0.99945214056507792,
  1.0129993658845911,
  1.0096736138419189,
  1.0210080677370397,
  1.0300722050305482,
  1.0324618908459047,
  1.0099466857643034,
  1.0007086056216046,
  0.99462449361171701],
 'Mean_Model_Score': 0.95285321759596642,
 'Mean_Wrong_Predictions': 9704.0}

{'Expert_Prediction_Percentage': [0.68523964708687701,
  0.68009758400881404,
  0.67689015691868759,
  0.67105263157894735,
  0.66506136398943605,
  0.66584727618745543,
  0.67155949052500774,
  0.66235175567785443,
  0.66322297879001335],
 'Mean_Model_Score': 0.95627621499943316,
 'Mean_Wrong_Predictions': 9000.0}

{'Expert_Prediction_Percentage': [0.99945214056507792,
  1.0129993658845911,
  1.0096736138419189,
  1.0210080677370397,
  1.0300722050305482,
  1.0324618908459047,
  1.0099466857643034,
  1.0007086056216046,
  0.99462449361171701],
 'Mean_Model_Score': 0.95285321759596642,
 'Mean_Wrong_Predictions': 9704.0}

{'Expert_Prediction_Percentage': [0.52881595477386933,
  0.50961014707026686,
  0.30342014433636649,
  0.26974358974358975,
  0.29972451790633609,
  0.31610614008022214,
  0.32840399590969871,
  0.25499244172169622,
  0.41326327521694134],
 'Mean_Model_Score': 0.95185455095089266,
 'Mean_Wrong_Predictions': 9910.0}

{'Expert_Prediction_Percentage': [0.26514549212911431,
  0.23381864219021997,
  0.67774555146194249,
  0.54040842072064965,
  0.38357696301621536,
  0.37549996078738923,
  0.56350526899611764,
  0.24940767651239931,
  0.54004683840749412],
 'Mean_Model_Score': 0.95418279378343507,
 'Mean_Wrong_Predictions': 9431.0}

# Unsupervised Learning
## Kmeans Cluster

In [ ]:
from sklearn.cluster import KMeans
clf_KMeans = KMeans(n_clusters=2, random_state=0).fit(X2)
display(clf_KMeans.labels_)

kmeans_predictions = clf_KMeans.predict(X2)
display(sum(clf_KMeans.predict(X2)))
display(sum(1 - clf_KMeans.predict(X2)))
display(clf_KMeans.cluster_centers_)

clf_kmeans_2d = clf_KMeans.transform(X2)

In [ ]:
clf_KMeans.labels_

In [ ]:
kmeans_x = clf_kmeans_2d[:,0]
kmeans_y = clf_kmeans_2d[:,1]
plt.scatter(kmeans_x, kmeans_y, c=[matplotlib.cm.spectral(float(i) /10) for i in clf_KMeans.labels_]);   
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2).fit(X)
pca_2d = pca.transform(X)
pca_x = pca_2d[:,0]
pca_y = pca_2d[:,1]
pl.scatter(pca_x, pca_y)
plt.show()


In [ ]:
pca_2d

In [ ]:
kmeans_predictions

In [ ]:
X_copy = X.assign(preds=pd.Series(kmeans_predictions).values)

In [ ]:
display(sum(X_copy.is_expert))
display(sum(X_copy.preds))
display(sum(X_copy.is_expert & X_copy.preds))